In [0]:
df=spark.read.format("json").option("multiline",True).load("/Volumes/usecase/json/pc-data-vm")

In [0]:
df.display()

In [0]:
df.columns

In [0]:
from pyspark.sql import functions as F

df_exploded = df.withColumn("claims", F.explode("claims"))
df_exploded.display()

In [0]:
df_flat = df_exploded.selectExpr(
    "claims.claim_id as claim_id",
    "claims.customer.contact as customer_contact",
    "claims.customer.id as customer_id",
    "claims.customer.name as customer_name",
    "claims.incident.date as incident_date",
    "claims.incident.description as incident_description",
    "claims.region as region",
    "claims.severity as severity",
    "claims.type as type"
)
df_flat = df_flat.withColumn("incident_date", F.try_to_date(df_flat["incident_date"], "yyyy-MM-dd"))
df_flat.display()

In [0]:
# from pyspark.sql.functions import explode,col
# df.printSchema()
# df_exploded= df.selectExpr("explode(claims.claim_id) as claim_id","explode(claims.type) as type","explode(claims.severity) as severity","explode(claims.customer) as customer","")
# df_exploded.display()


In [0]:
from pyspark.sql.functions import count,col
count=df_flat.select(col('claim_id')).distinct().count()
print(count)

In [0]:
from pyspark.sql.functions import count
claims_count=df_flat.groupBy('type').agg(count('claim_id').alias('count_claims'))
claims_count.display()

In [0]:
from pyspark.sql.functions import count
severity_count = df_flat.groupBy('region').agg(count('claim_id').alias('count_region'))
severity_count.display()

In [0]:
regional_claim_count = df_flat.groupBy('region').agg(count('claim_id').alias('count_region'))
regional_claim_count.display()

In [0]:
from pyspark.sql.functions import count,avg,when,round

avg_regional_severity = df_flat.withColumn("severity_value",when(col('severity')=='High',3).when(col('severity')=='Medium',2).when(col('severity')=='Low',1).otherwise(0)).groupBy("region").agg(round(avg('severity_value'),1).alias('avg_severity')).withColumn("Average_severity",when(col('avg_severity')>=3.0,'High').when((col('avg_severity')>=2.0) & (col('avg_severity')<3.0),'Medium').when((col('avg_severity')>=1.0) & (col('avg_severity')<2.0),'Low').otherwise('None')).drop('avg_severity')
avg_regional_severity.display()